In [1]:
import numpy as np
import matplotlib.pyplot as plt

coll_points = np.load("./data/sampling_points.npz")["coll_points"]
bc1_points = np.load("./data/sampling_points.npz")["bc1_points"]
bc2_points = np.load("./data/sampling_points.npz")["bc2_points"]
bc3_points = np.load("./data/sampling_points.npz")["bc3_points"]
bc4_points = np.load("./data/sampling_points.npz")["bc4_points"]

In [2]:
import torch

# ------------------------------- #
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available.")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    device = torch.device("cpu")
    print("CPU is used.")

coll_points = torch.tensor(coll_points, dtype=torch.float32, device=device, requires_grad=True)
bc1_points = torch.tensor(bc1_points, dtype=torch.float32, device=device, requires_grad=True)
bc2_points = torch.tensor(bc2_points, dtype=torch.float32, device=device, requires_grad=True)
bc3_points = torch.tensor(bc3_points, dtype=torch.float32, device=device, requires_grad=True)
bc4_points = torch.tensor(bc4_points, dtype=torch.float32, device=device, requires_grad=True)

CPU is used.


In [3]:
import torch
import torch.nn as nn
from mlp import MLP
import time
import loss_functions

# ------------------------------- #
# Training
u_hat = MLP(input_dim = 2, hidden_dim = 5, num_layers = 5, output_dim = 1, activation = "tanh").to(device)

total_params = sum(p.numel() for p in u_hat.parameters())
weight_params = sum(p.numel() for name, p in u_hat.named_parameters() if 'weight' in name)
bias_params = sum(p.numel() for name, p in u_hat.named_parameters() if 'bias' in name)

print("Model summary:")
print(f"- Number of total parameters: {total_params}")
print(f"- Weight: {weight_params}")
print(f"- Bias: {bias_params}\n")

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

u_hat.apply(init_weights)

iterations = 20000
learning_rate = 1e-3

optimizer = torch.optim.Adam(u_hat.parameters(), lr=learning_rate)
# optimizer = torch.optim.LBFGS(u_hat.parameters(), lr = learning_rate, 
#                             max_iter = 500, max_eval = 500, history_size = 50,
#                             line_search_fn = 'strong_wolfe',
#                             tolerance_grad = 1e-8, tolerance_change = 1e-10)

total_loss_list = []
pde_loss_list = []
bc1_loss_list = []
bc2_loss_list = []
bc3_loss_list = []
bc4_loss_list = []

print("Training start...")
start_time = time.time()

for epoch in range(1, iterations + 1):
    optimizer.zero_grad()
    
    total_loss = loss_functions.total_loss(u_hat, coll_points, bc1_points, bc2_points, bc3_points, bc4_points)
    
    total_loss.backward()
    
    optimizer.step()

    pde_loss = loss_functions.pde_loss(u_hat, coll_points)
    bc1_loss = loss_functions.bc1_loss(u_hat, bc1_points)
    bc2_loss = loss_functions.bc2_loss(u_hat, bc2_points)
    bc3_loss = loss_functions.bc3_loss(u_hat, bc3_points)
    bc4_loss = loss_functions.bc4_loss(u_hat, bc4_points)

    total_loss_list.append(total_loss.detach().item())
    pde_loss_list.append(pde_loss.detach().item())
    bc1_loss_list.append(bc1_loss.detach().item())
    bc2_loss_list.append(bc2_loss.detach().item())
    bc3_loss_list.append(bc3_loss.detach().item())
    bc4_loss_list.append(bc4_loss.detach().item())

    if epoch % 100 == 0:
        print(f"Epoch {epoch:05d}: "
              f"Total loss = {total_loss.detach().item():.2e}, "
              f"pde loss = {pde_loss.detach().item():.2e}, "
              f"bc1 loss = {bc1_loss.detach().item():.2e}, "
              f"bc2 loss = {bc2_loss.detach().item():.2e}, "
              f"bc3 loss = {bc3_loss.detach().item():.2e}, "
              f"bc4 loss = {bc4_loss.detach().item():.2e}")

end_time = time.time()
elapsed_time = end_time - start_time

print("Training complete!\n")
print(f"Total training time: {elapsed_time:.2f} seconds")

np.savez("./results/loss_data.npz",
        total_loss=np.array(total_loss_list),
        pde_loss=np.array(pde_loss_list),
        bc1_loss=np.array(bc1_loss_list),
        bc2_loss=np.array(bc2_loss_list),
        bc3_loss=np.array(bc3_loss_list),
        bc4_loss=np.array(bc4_loss_list))

torch.save(u_hat.state_dict(), "./results/model.pth")

Model summary:
- Number of total parameters: 141
- Weight: 115
- Bias: 26

Training start...
Epoch 00100: Total loss = 5.02e-01, pde loss = 2.59e-02, bc1 loss = 6.52e-02, bc2 loss = 3.39e-01, bc3 loss = 6.61e-02, bc4 loss = 5.23e-03
Epoch 00200: Total loss = 4.46e-01, pde loss = 4.62e-02, bc1 loss = 7.67e-02, bc2 loss = 2.46e-01, bc3 loss = 7.70e-02, bc4 loss = 9.90e-05
Epoch 00300: Total loss = 4.06e-01, pde loss = 3.39e-02, bc1 loss = 7.86e-02, bc2 loss = 2.10e-01, bc3 loss = 8.10e-02, bc4 loss = 1.24e-03
Epoch 00400: Total loss = 3.65e-01, pde loss = 2.25e-02, bc1 loss = 7.74e-02, bc2 loss = 1.71e-01, bc3 loss = 8.96e-02, bc4 loss = 4.20e-03
Epoch 00500: Total loss = 3.37e-01, pde loss = 2.04e-02, bc1 loss = 7.32e-02, bc2 loss = 1.40e-01, bc3 loss = 9.68e-02, bc4 loss = 6.42e-03
Epoch 00600: Total loss = 3.12e-01, pde loss = 2.26e-02, bc1 loss = 6.94e-02, bc2 loss = 1.21e-01, bc3 loss = 9.33e-02, bc4 loss = 5.99e-03
Epoch 00700: Total loss = 2.87e-01, pde loss = 2.60e-02, bc1 loss =